# Prep data for modeling by pulling current season in realtime
### NOTE:  this requires an updated history file, which must be manually refreshed each season)

In [ ]:
#Only run this code if you need to update the history file
from jobs import SRjobs
SRjobs.create_history()

### Run the next section every time you add new feature engineering, so both past/future games will get it

In [1]:
from jobs import SRjobs, SRfeatures
import pandas as pd
schedule_mstr = SRjobs.get_season_played_games()
future_mstr = SRjobs.get_season_future_games()
history = SRjobs.get_history()

Initiate get_season_played_games
Finished scraping 2018 schedule with 845 rows
Successfully loaded 828.0 played games from 2018
Initiate get_season_future_games
Finished scraping 2018 schedule with 845 rows
Successfully loaded future_mstr with 17 remaining games in 2018


In [2]:
full_history = pd.concat([history, schedule_mstr])
print('Applying features to games already played')
full_history = SRfeatures.apply_features(full_history)
all_time = pd.concat([full_history, future_mstr], sort=True)
print('Applying features to future games')
future_mstr = SRfeatures.apply_future_features(future_mstr, all_time)

Applying features to games already played
Added Game_conf
Added Team_SRTD
Added Team_CRTD
Added Opp_SRTD
Added Opp_CRTD
Added Team_OffStrat
Added Opp_OffStrat
Added Team_DefStren
Added Opp_DefStren
Applying features to future games
Added Game_conf
Added Team_SRTD
Added Team_CRTD
Added Opp_SRTD
Added Opp_CRTD
Added Team_OffStrat
Added Opp_OffStrat
Added Team_DefStren
Added Opp_DefStren


### Normalize any features which require it

In [6]:
def nz_num(num, feature):
    min_DFS = min(full_history[feature])
    max_DFS = max(full_history[feature])
    return (num-min_DFS)/(max_DFS - min_DFS)

def nz_features(feature_list):
    for feature in feature_list:
        
        full_history[feature + '_nz'] = full_history.apply(lambda x: 
                                                      nz_num(x[feature], feature), axis=1)
        print('Normalized ' + feature + ' on full_history')
        future_mstr[feature + '_nz'] = future_mstr.apply(lambda x: 
                                                      nz_num(x[feature], feature), axis=1)
        print('Normalized ' + feature + ' on future_mstr')
        full_history.drop(feature, axis=1, inplace=True)
        future_mstr.drop(feature, axis=1, inplace=True)

feature_list = ['Team_OffStrat', 'Opp_OffStrat', 'Team_DefStren', 'Opp_DefStren']

nz_features(feature_list)

Normalized Team_OffStrat on full_history
Normalized Team_OffStrat on future_mstr
Normalized Opp_OffStrat on full_history
Normalized Opp_OffStrat on future_mstr
Normalized Team_DefStren on full_history
Normalized Team_DefStren on future_mstr
Normalized Opp_DefStren on full_history
Normalized Opp_DefStren on future_mstr


In [8]:
full_history.to_csv('./SRinput/current/past_games.csv')

In [9]:
future_mstr.to_csv('./SRinput/current/future_games.csv')